In [11]:
import os
from dotenv import load_dotenv


load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [12]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

In [13]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

result

2024-10-28 16:17:18,063 - 8440214080 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
I need to come up with engaging and viral video ideas for a YouTube channel in the Hot Sauce industry that would appeal to a young audience. These ideas should be creative and have the potential to generate significant interest and shares.

Action: Delegate work to co-worker  
Action Input: {"co-worker": "Senior Researcher", "task": "Research trending topics, challenges, and popular formats in the food and hot sauce niche on YouTube. Focus on what resonates with younger audiences, looking for specific examples of successful videos and themes.","context": "The goal is to gather insights into current trends in the hot sauce niche that could inspire video ideas. This includes understanding what type of content is performing well, what challenges are popular, and how to incorporate humor or unique angle

'### Title: **The Ultimate One Chip Challenge: Spicy Trivia Showdown!**\n\n---\n\n**[INTRO]**\n\n[Opening Scene: Energetic music plays with fast-paced cuts of people reacting to the One Chip Challenge, spicy peppers, and trivia questions.]\n\n**Host (on-screen):**  \n"Welcome back to our channel! Today, we’re diving into one of the hottest trends on the internet—the Ultimate One Chip Challenge! But we’re not just stopping there. We’re spicing things up with a trivia showdown that will test your knowledge on all things spicy! Are you ready to burn and learn? Let’s get started!"\n\n---\n\n**[SEGMENT 1: The One Chip Challenge Explained]**\n\n[Cut to visuals of the One Chip Challenge packaging and footage of participants.]\n\n**Host (voiceover):**  \n"The One Chip Challenge, created by Paqui, involves eating just one tortilla chip made from some of the hottest peppers in the world—the Carolina Reaper and the Naga Viper. These bad boys clock in at over 1.7 million Scoville heat units! That’